# DNA File Compression Analysis
Ian Switzer, Joshua Devine, Ronith Ranjan

May 2, 2022

## Overview

This project was undertaken a part of CS 4501: Computational Biology taught at the University of Virginia by Professor David Evans. Our goal was to examine existing DNA/RNA formats in the field of computational biology to find more efficient ways to store genetic information. Next Generation Sequencing (NGS) has allowed for massive amounts of genomics data to be produced, which has shifted the bottleneck from sequencing to computation; specifically long term storing, managing, and analyzing large amounts of data. Compression tools can reduce the necessary amount of storage. 

Our project subgoals were as follows: 
1. Develop our own compressed DNA file format as a learning tool
2. Compare our new file format with existing file formats
3. Identify the file format with the best performance metrics
4. Build a standalone Python library for that file format
5. Add support for that file format in [BioPython](https://biopython.org)

## File Format Development

Each of our approaches to file compression were based around two core ideas:
1. Using ASCII characters to represent bases results in many wasted bits of information. Representing 'A', 'T', 'G', or 'C' requires as few as two bits, whereas ASCII requires eight.
2. Pairs of adjacent bases or reads can be further compressed by using a lookup table of the most common pairs.

### .dna (fasta)

> parse_fasta.py

In this approach, we scan a fasta file for adjacent bases and determine the most common pairs. These are encoded into a table at the beginning of the output file. The sequence is then written to the output file in-order, with groupings of four bits representing either:
- an individual base, or
- a pair of bases in the lookup table

### .dna (fastq)

> parse_fastq.py

Unlike fasta files, fastq files also contain quality scores for each read. In order to store this data, we adjusted our pairing algorithm to determine the most common pairs of base and quality score.  Again, these were encoded into a table at the beginning of the output file. The sequence was then written to the output file in-order, with groupings of four bits representing either:
- an individual base, or
- a pair of a base and a quality score
In the case where we read only an individual base and not a pair, the following eight bits encode the base's respective quality score. As for the identifiers of each read, they are stored sequentially at the end of the file in ASCII.

### .dna_seg

> parse_fastq_seg.py

This approach is an addition to the prior approach, with the exact same compression scheme regarding read/score data. However, in this approach, we realize that we have an uncompressed segment of text at the end of the file. Because of gzip's ability to compress general text data, we used it to compress this section alone.

### .dna_37

> parse_fastq_37.py

Similar to our initial fastq approach, the three most common pairs of base and quality score were encoded into a table at the beginning of the output file. The sequence was then written to the output file in-order, with groupings of three bits representing either:
- an individual base, or
- a pair of a base and a quality score
In the case where we read only an individual base and not a pair, the following seven bits encode the base's respective quality score. As for the identifiers of each read, they are stored sequentially at the end of the file in ASCII.

## Comparison Testing

We identified four state-of-the-art FASTQ compression schemes (via [UCSC](http://genome.ucsc.edu/FAQ/FAQformat.htm), [SCFBM](https://scfbm.biomedcentral.com/articles/10.1186/s13029-019-0073-5)): 
- [Leon](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-015-0709-7) [(code)](http://gatb.inria.fr/software/leon/)
    - Constructs a de Bruijn graph from the FASTQ reads and encodes each read as a part within the graph
- [Lfqc](https://academic.oup.com/bioinformatics/article/31/20/3276/196127?login=true) [(code)](https://github.com/mariusmni/lfqc)
    - Lossless compression scheme; advanced read-identifier tokenization scheme
    - PAQ family members for compression of read sequences and quality scores
- [Scalce](https://academic.oup.com/bioinformatics/article/28/23/3051/195414?login=true) [(code)](http://sfu-compbio.github.io/scalce/)
    - Rearranges reads to offer high compression rate and speed without a reference genome and irrespective of the compression algorithm used
    - Boosting scheme (ML process) that uses locally-consistent parsing technique
- [Slimfastq](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0059190) [(code)](https://github.com/Infinidat/slimfastq)
    - Robust re-implementation of another FASTQ algorithm: Fqzcomp
    - One of the fastest compression algorithms that provides reasonable compression rates


These schemes were tested for their compression ratios across a span of multi-read fastq files. The sizes of these files ranged from 1062 bytes to 5875259 bytes with varied read complexity. Included in the tests were also our newly created file formats and the standard [gzip]() compression format.

### Results

#### File Size (bytes)

| | fastq | dna | dna_37 | dna_seg | gzip | leon | lfqc | scalce | slimfastq |
|-|-|-|-|-|-|-|-|-|-|
|tst7|1062|386|345|278|170|1314|10240|2048279|49152|
|tstb|1248|718|767|445|512|1428|10240|2048443|49152|
|tstc|22448|ERROR|ERROR|ERROR|6312|3430|10240|ERROR|49152|
|tst6|50455|29905|30092|19594|16696|6879|20480|2060009|40960|
|tsta|50455|29905|30092|19594|16696|6879|20480|2060009|40960|
|tst4|629368|425445|414513|323275|247351|73113|174080|2225114|221184|
|tst3|638329|399756|409686|295414|266395|75946|163840|2239480|253952|
|tst2|643053|411263|410948|305822|218435|71578|143360|2195847|196608|
|tst1|656399|441174|434344|322899|228760|71211|153600|2204329|204800|
|tst5|672867|422213|438206|282080|205747|71045|133120|2186228|180224|
|tst8|797964|ERROR|ERROR|ERROR|358440|111195|296960|ERROR|385024|
|tst9|5875259|3246879|3454900|2671921|1656443|510707|880640|3124541|1015808|

#### File Size (%)

| | fastq | dna | dna_37 | dna_seg | gzip | leon | lfqc | scalce | slimfastq |
|-|-|-|-|-|-|-|-|-|-|
|tst7|100.00%|36.35%|32.49%|26.18%|16.01%|123.73%|ERROR|ERROR|ERROR|
|tstb|100.00%|57.53%|61.46%|35.66%|41.03%|114.42%|ERROR|ERROR|ERROR|
|tstc|100.00%|ERROR|ERROR|ERROR|28.12%|15.28%|45.62%|ERROR|ERROR|
|tst6|100.00%|59.27%|59.64%|38.83%|33.09%|13.63%|40.59%|ERROR|81.18%|
|tsta|100.00%|59.27%|59.64%|38.83%|33.09%|13.63%|40.59%|ERROR|81.18%|
|tst3|100.00%|62.63%|64.18%|46.28%|41.73%|11.90%|25.67%|ERROR|39.78%|
|tst4|100.00%|63.95%|63.91%|47.56%|33.97%|11.13%|22.29%|ERROR|30.57%|
|tst2|100.00%|63.95%|63.91%|47.56%|33.97%|11.13%|22.29%|ERROR|30.57%|
|tst1|100.00%|67.21%|66.17%|49.19%|34.85%|10.85%|23.40%|ERROR|31.20%|
|tst5|100.00%|62.75%|65.13%|41.92%|30.58%|10.56%|19.78%|ERROR|26.78%|
|tst8|100.00%|ERROR|ERROR|ERROR|44.92%|13.93%|37.21%|ERROR|48.25%|
|tst9|100.00%|55.26%|58.80%|45.48%|28.19%|8.69%|14.99%|53.18%|17.29%|

#### Analysis
- Our attempts:
    - Compression errors occurred in 3 test cases
    - Beat gzip in only a single test case
    - Beat scalce in every test case
    - Better than other formats in ~33% of matchups
- Overall:
    - Leon compressed files over twice as small as any other format
    - Gzip was better than most other formats
    - Many of the formats resulted in increases of over 200% in filesize (included in ERROR data)



## Standalone Library

With Leon outperforming the other formats, we placed our focus on making the Leon library (which is C++-based) available to Python users. This led to the creation of [PyLeon](https://github.com/jdvne/pyleon), a module wrapping Python around Leon. This allows users to compress/decompress `.leon` files from within a Python environment for use in data processing.

> https://github.com/jdvne/pyleon

## Pull Request

BioPython is the most popular of the open-source tools available for analyizing genome data in Python. It currently has modules for opening many of the common (and some uncommon) dna file formats. Adding support for `.leon` files would allow researchers to seamlessly open, process, and store genomic data from a highly-compressed form. We added this support in a [Pull Request](https://github.com/biopython/biopython/pull/3919) to the BioPython library.

> https://github.com/biopython/biopython/pull/3919

> https://github.com/jdvne/biopython